In [84]:
# "Sanaz Hosseini"- University of North Carolina at Charlotte
# Introduction to Machine Learning Class - Instructor: Prof. Hamed Tabkhi
# Pytorch & Neural Networks
# Homework 5

In [60]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(edgeitems=2, linewidth=75)

In [61]:
df = pd.read_csv('Housing.csv')

In [62]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [63]:
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = df[num_vars]
df_new.head()

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2


In [64]:
df_new.shape

(545, 6)

In [65]:
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

C:\Users\Sanaz\AppData\Local\Temp\ipykernel_20436\2943118898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[num_vars] = scaler.fit_transform(df_new[num_vars])


,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [66]:
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values
X.shape

(545, 5)

In [67]:
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape

torch.Size([545, 5])

In [68]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_X = X[train_indices]
train_Y = Y[train_indices]
train_Y = train_Y.unsqueeze(1)

val_X = X[val_indices]
val_Y = Y[val_indices]
val_Y = val_Y.unsqueeze(1)

In [69]:
print(train_X.shape)
print(train_Y.shape)
print(val_X.shape)
print(val_Y.shape)

torch.Size([436, 5])
torch.Size([436, 1])
torch.Size([109, 5])
torch.Size([109, 1])


In [70]:
train_X = train_X.float()
train_Y = train_Y.float()
val_X = val_X.float()
val_Y = val_Y.float()
print(train_X.dtype)
print(train_Y.dtype)
print(val_X.dtype)
print(val_Y.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


# Building a Neural Network with one hidden layer : 

In [71]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(5, 8)),
    ('hidden_activation', nn.ReLU()), 
    ('output_linear', nn.Linear(8, 1))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=5, out_features=8, bias=True)
  (hidden_activation): ReLU()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [72]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [73]:
seq_model.output_linear.bias

Parameter containing:
tensor([-0.1320], requires_grad=True)

In [74]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_X, val_X, train_Y, val_Y):
    
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_X) 
        train_loss = loss_fn(train_t_p, train_Y)
                             
        val_t_p = model(val_X) 
        val_loss = loss_fn(val_t_p, val_Y)
        
        optimizer.zero_grad()
        train_loss.backward() 
        optimizer.step()

        if epoch ==1 or epoch % 100==0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

In [75]:
optimizer= optim.SGD(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)


Epoch 1, Training loss 0.2006, Validation loss 0.2141
Epoch 100, Training loss 0.0231, Validation loss 0.0275
Epoch 200, Training loss 0.0218, Validation loss 0.0258


In [76]:
optimizer= optim.Adam(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)


Epoch 1, Training loss 0.0218, Validation loss 0.0257
Epoch 100, Training loss 0.0106, Validation loss 0.0135
Epoch 200, Training loss 0.0101, Validation loss 0.0130


In [77]:
optimizer= optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)


Epoch 1, Training loss 0.0101, Validation loss 0.0129
Epoch 100, Training loss 0.0101, Validation loss 0.0129
Epoch 200, Training loss 0.0101, Validation loss 0.0129


# Extending the Neural Network with one more additional hidden layer : 

In [79]:
seq_model = nn.Sequential(
            nn.Linear(5, 16), 
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1))

seq_model

Sequential(
  (0): Linear(in_features=5, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
)

In [80]:
optimizer= optim.Adam(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)

Epoch 1, Training loss 0.1213, Validation loss 0.1274
Epoch 100, Training loss 0.0109, Validation loss 0.0145
Epoch 200, Training loss 0.0107, Validation loss 0.0146


# Extending the Neural Network with two more additional hidden layers : 

In [82]:
seq_model = nn.Sequential(
            nn.Linear(5, 17), 
            nn.ReLU(),
            nn.Linear(17, 12),
            nn.ReLU(),
            nn.Linear(12, 8),
            nn.ReLU(),
            nn.Linear(8, 1))

seq_model

Sequential(
  (0): Linear(in_features=5, out_features=17, bias=True)
  (1): ReLU()
  (2): Linear(in_features=17, out_features=12, bias=True)
  (3): ReLU()
  (4): Linear(in_features=12, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)

In [83]:
optimizer= optim.Adam(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)

Epoch 1, Training loss 0.0287, Validation loss 0.0325
Epoch 100, Training loss 0.0091, Validation loss 0.0128
Epoch 200, Training loss 0.0086, Validation loss 0.0134
